## Fully Connected layer

In [ ]:
#%matplotlib inline
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
t_max = 2**9
dims = (1,120,1)
n_outputs = 84
weight_e = 500 # 101 # 
weight_acc = 2**8
weight_acc_real = weight_acc / 2
model_args = {'weight_e':weight_e, 'weight_acc':weight_acc}

np.random.seed(seed=47)
weights = (np.random.rand(n_outputs,np.product(dims)) - 0.5) / 4
biases = (np.random.rand(n_outputs) - 0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims, **model_args),
    layers.Dense(weights=weights, biases=None, **model_args),
    layers.MonitorLayer(**model_args),
])

calc_probe = quartz.probe(loihi_model.layers[1].blocks[-3].neurons[0])
sync_probe = quartz.probe(loihi_model.layers[1].blocks[-3].neurons[2])
input_probe = quartz.probe(loihi_model.layers[0].blocks[0])
hidden_probe9 = quartz.probe(loihi_model.layers[1].blocks[-3])
hidden_probe0 = quartz.probe(loihi_model.layers[1].blocks[0])
output_probe = quartz.probe(loihi_model.layers[2].blocks[0])
bias9_block = loihi_model.layers[1].blocks[-2]
bias9_probe = quartz.probe(bias9_block)
bias7_block = loihi_model.layers[1].blocks[-8]
bias7_probe = quartz.probe(bias7_block)

values = np.random.rand(np.product(dims)) # np.ones((np.product(dims))) * 0.5
inputs = quartz.decode_values_into_spike_input(values, t_max)

quantized_values = (values*t_max).round()/t_max
quantized_weights = (weight_acc_real*weights).round()/weight_acc_real
quantized_biases = (biases*t_max).round()/t_max

pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dims), out_features=n_outputs), 
    nn.ReLU()
)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor(np.zeros_like(quantized_biases)))
model_output = pt_model(torch.tensor(quantized_values)).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:

np.array([value[0] for (key, value) in sorted(output[0].items())])

In [ ]:
hidden_probe9.plot()

## Convolutional Layer

In [ ]:
import quartz
import numpy as np
import torch
import torch.nn as nn
t_max = 2**8
run_time = 8*t_max
input_dims=   (6, 5,  5, 2)
weight_dims = (32,6,5,5)
weight_e = 500
weight_acc = 128
kernel_size = weight_dims[2:]

l0 = quartz.layers.InputLayer(dims=input_dims, monitor=False, weight_e=weight_e, weight_acc=weight_acc)
weights = (np.random.rand(*weight_dims)-0.5) / 5
biases = (np.random.rand(weight_dims[0])-0.5) / 2
l1 = quartz.layers.Conv2D(prev_layer=l0, weights=weights, biases=biases, split_output=False,\
                         monitor=False, weight_e=weight_e, weight_acc=weight_acc)
l2 = quartz.layers.MonitorLayer(prev_layer=l1, weight_e=weight_e, weight_acc=weight_acc)

values = np.random.rand(np.product(input_dims)//2) / 2
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max
quantized_values = quantized_values.reshape(*input_dims[:3])
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=kernel_size), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = torch.nn.Parameter(torch.tensor(quantized_biases))
model_output = model(torch.tensor(values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()

In [ ]:
output_values, spike_times = l2.run_on_loihi(run_time, t_max=t_max, input_spike_list=inputs, probe_selection=["sync", "calc"], plot=False)
output_combinations = list(zip([value[0] for (key, value) in sorted(output_values.items())], model_output.flatten()))
output_combinations

## Max Pooling layer

In [ ]:
import quartz
import numpy as np
t_max = 2**8
run_time = 8*t_max
input_dims = (1,10,10,2)
kernel_size = [2,2]

l0 = quartz.layers.InputLayer(dims=input_dims, monitor=False)
l1 = quartz.layers.MaxPool2D(prev_layer=l0, kernel_size=kernel_size, split_output=True, monitor=False)
l2 = quartz.layers.MonitorLayer(prev_layer=l1)

np.random.seed(seed=45)
values = np.random.rand(np.product(dims)//2)
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).astype(int)/t_max

model = nn.Sequential(nn.MaxPool2d(kernel_size=kernel_size, stride=stride), nn.ReLU())
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()

In [ ]:
output_values, spike_times = l2.run_on_loihi(run_time, t_max=t_max, input_spike_list=inputs, plot=False)
output_values